## Data Engineer Co-op Case Study for Snapcommerce

##### By Jiayun Zhu

In [1]:
import pandas as pd
import io
import string
from pandasql import sqldf

In [2]:
data = 'Airline Code;DelayTimes;FlightCodes;To_From\nAir Canada (!);[21, 40];20015.0;WAterLoo_NEWYork\n<Air France> (12);[];;Montreal_TORONTO\n(Porter Airways. );[60, 22, 87];20035.0;CALgary_Ottawa\n12. Air France;[78, 66];;Ottawa_VANcouvER\n""".\\.Lufthansa.\\.""";[12, 33];20055.0;london_MONTreal\n'

In [3]:
data = io.StringIO(data)

In [4]:
# coverting the string to a dataframe
df = pd.read_csv(data, sep=";" )
df

,Airline Code,DelayTimes,FlightCodes,To_From
0,Air Canada (!),"[21, 40]",20015.0,WAterLoo_NEWYork
1,<Air France> (12),[],NaN,Montreal_TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035.0,CALgary_Ottawa
3,12. Air France,"[78, 66]",NaN,Ottawa_VANcouvER
4,""".\.Lufthansa.\.""","[12, 33]",20055.0,london_MONTreal


In [5]:
## FlightCodes missing data handling
flightCodes_idx = df.columns.get_loc("FlightCodes")
start_point = int(df.iloc[0,flightCodes_idx])
n_increment = 10

df.FlightCodes = list(range(start_point, start_point + n_increment * len(df),n_increment))

In [6]:
## To_From
df[["Departure", "Arrival"]] = df["To_From"].str.upper().str.split("_",expand = True)
df = df.drop(columns="To_From")

In [7]:
# remove punctuation
str_punct = string.punctuation

def remove_punct(s):
    for punct in str_punct:
        s = s.replace(punct, '')
    return s

df["Airline Code"] = df["Airline Code"].apply(remove_punct)

In [8]:
# all flights leaving from Waterloo
departure_city = "WATERLOO"
query = '''select * 
        from df
        where departure = "{var1}"
        ;
        '''.format(var1 = departure_city.upper())
results = sqldf(query)